In [1]:
import pandas as pd
import pymongo
import numpy as np

In [2]:
client = pymongo.MongoClient()
db = client.swatford

In [3]:
generif = pd.read_table("../data/pubmed/generifs_basic.gz",comment="#",header=None,names=["tax_id","gene_id",
                                                                                       "pmid","ts","txt"],
                     dtype={"pmid":str})

In [4]:
generif.pmid = generif.pmid.apply(lambda x: [int(y) for y in x.split(",")])

In [9]:
data = {"tax_id":[],"gene_id":[],"pmid":[]}

In [10]:
for row in generif.itertuples():
    data["tax_id"]+=[int(row.tax_id)]*len(row.pmid)
    data["gene_id"]+=[int(row.gene_id)]*len(row.pmid)
    data["pmid"]+=row.pmid

In [15]:
to_insert = []
for i,row in pd.DataFrame(data).drop_duplicates().iterrows():
    to_insert.append({k:int(v) for k,v in dict(row).items()})
    if i%25000==0:
        db.generif.insert_many(to_insert)
        to_insert=[]
db.generif.insert_many(to_insert)